In [2]:
import base64
import requests
import os
import autogen
import pprint


In [3]:
print(dir(autogen))
print(autogen.__file__)


['Agent', 'AssistantAgent', 'ChatCompletion', 'Completion', 'ConversableAgent', 'DEFAULT_MODEL', 'FAST_MODEL', 'GroupChat', 'GroupChatManager', 'UserProxyAgent', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'agentchat', 'code_utils', 'config_list_from_dotenv', 'config_list_from_json', 'config_list_from_models', 'config_list_gpt4_gpt35', 'config_list_openai_aoai', 'get_config_list', 'logger', 'logging', 'oai', 'version']
/Users/serkanhaytac/experimental-projects/autogen-habittracker/venv/lib/python3.11/site-packages/autogen/__init__.py


In [4]:
config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)
print(config_list_gpt4)


[]


In [5]:
# OpenAI API Key
api_key = os.getenv("OPENAI_API_KEY")


In [6]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
def encode_image_to_base64(image_path):
    # This function reads the image file and converts it to a base64 string
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode()
    return encoded_string

def send_image_to_api(image_path):
    # Encode the image to base64
    base64_image = encode_image_to_base64(image_path)
  

    # Prepare headers
    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }

    # Prepare payload
    payload = {
      "model": "gpt-4-vision-preview",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
              }
            }
          ]
        }
      ],
      "max_tokens": 1000
    }

    # Send the request
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return pprint.pprint(response.json())


In [7]:


llm_config = {
    "config_list": config_list_gpt4,
    "seed": 5,
    "temperature": 0,
    "functions": [
        {
            "name": "send_image_to_api",
            "description": "Gets image path and sends it to the API.",
            "parameters": {
                "type": "object",
                "properties": {
                    "image_path": {
                        "type": "string",
                        "description": "Valid path.",
                    }
                },
                "required": ["image_path"],
            },
        }
    ]
}
engineer = autogen.AssistantAgent(
    name="Assistant",  # the default assistant agent is capable of solving problems with code
    llm_config=llm_config,
    system_message='''Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the 
    code in a code block that specifies the script type. The user can't modify your code. So do not suggest 
    incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by 
    the executor. Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. 
    Check the execution result returned by the executor. If the result indicates there is an error, fix the error and 
    output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed 
    or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your 
    assumption, collect additional info you need, and think of a different approach to try.''',)

planner =  autogen.AssistantAgent(
    name="Planner",
    system_message='''Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin 
    approval. The plan may involve an engineer who can write code and an executor and critic who doesn't write code. 
    Explain the plan first. Be clear which step is performed by an engineer, executor, and critic.''',
    llm_config=llm_config,
)

executor = autogen.AssistantAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    code_execution_config={"last_n_messages": 3, "work_dir": "feedback"},
)

critic =  autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback.",
    llm_config=llm_config,
)

user_proxy = autogen.AssistantAgent(
   name="Admin",
   system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved "
                  "by this admin.",
   code_execution_config=False,
)

user_proxy.register_function(function_map={
    "send_image_to_api": send_image_to_api
})


In [8]:
# user_proxy.initiate_chat(
#     coder,
#     message="""Upload /Users/serkanhaytac/experimental-projects/autogen-habittracker/app_screenshot.jpg
#     image using send_image_to_api registered function. Once image is uploaded, come up with a plan to create a habit tracker app.
#     App should be react native app that has the ability to keep track of habits"
#     """
# )
groupchat = autogen.GroupChat(agents=[user_proxy, engineer, planner, executor, critic], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)
user_proxy.initiate_chat(manager, message="""Create a menu bar app that has the ability to take a openai api key to track open ai spending  """)


Admin (to chat_manager):

Create a menu bar app that has the ability to take a openai api key to track open ai spending  

--------------------------------------------------------------------------------
[autogen.oai.completion: 11-19 09:41:25] {788} WARNING - Completion was provided with a config_list, but the list was empty. Adopting default OpenAI behavior, which reads from the 'model' parameter instead.
[autogen.oai.completion: 11-19 09:41:25] {788} WARNING - Completion was provided with a config_list, but the list was empty. Adopting default OpenAI behavior, which reads from the 'model' parameter instead.
Planner (to chat_manager):

Plan:

1. **Engineer**: Design and develop a menu bar application. This application will have a form to input the OpenAI API key. The application will also have a feature to display the current spending on OpenAI.

2. **Engineer**: Implement a function to validate the OpenAI API key. This function will check if the key is in the correct format and if i